Descobrindo novos materiais usando Inteligência Artificial (e aprendendo algo novo no processo)
===============================================================================================

**Autor:** Daniel R. Cassar



## Parte 0 - Avisos, instalações e importações



1.  Estrutura do notebook

2.  Python

3.  Comentários e dúvidas no final de cada parte

4.  Módulos básicos: `numpy`, `scipy`, `pandas` e `matplotlib`

5.  Módulos específicos: `scikit-learn`, `pymoo`, `shap` e `glasspy`



In [ ]:
import sys

!{sys.executable} -m pip install pymoo==0.6.0.1
!{sys.executable} -m pip install shap==0.42.1
!{sys.executable} -m pip install glasspy==0.4.4

In [ ]:
import numpy as np
import pandas as pd
import shap
from matplotlib import pyplot as plt
from scipy.stats import randint

from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error
from sklearn import tree
from sklearn.model_selection import RandomizedSearchCV
from sklearn.tree import export_text

from pymoo.core.problem import ElementwiseProblem, Problem
from pymoo.algorithms.moo.nsga2 import NSGA2
from pymoo.operators.crossover.sbx import SBX
from pymoo.operators.mutation.pm import PM
from pymoo.operators.sampling.rnd import FloatRandomSampling
from pymoo.termination import get_termination
from pymoo.optimize import minimize

from glasspy.predict import GlassNet
from glasspy.chemistry import to_array
from glasspy.data import SciGlass

<hr>



## Parte 1 - Dados



Banco de dados SciGlass

1.  Histórico / criadores

2.  Estatísticas

3.  Licença Open Database [https://github.com/epam/SciGlass](https://github.com/epam/SciGlass)



### Carregando dados do SciGlass usando `GlassPy`



O módulo `GlassPy` já tem os dados do SciGlass e já carrega eles em DataFrames de `pandas`.

Na primeira vez que rodamos temos que baixar os dados do servidor, isso pode demorar uns minutos. Fora isso, toda vez que você carregar os dados, o `GlassPy` deve ler as tabelas do SciGlass e processá-las. Isto também leva um certo tempo.



In [ ]:
sciglass = SciGlass()

df = sciglass.data
df

### Informações disponíveis e seu significado



Metadados:

-   `ChemicalAnalysis`: Indicates if the glass composition was obtained by chemical analysis
-   `Author`: First author of the publication
-   `Year`: Year of the publication

Propriedades:

-   `T0`: Temperature where viscosity is 1 Pa.s (K)
-   `T1`: Temperature where viscosity is 10 Pa.s (K)
-   `T2`: Temperature where viscosity is 100 Pa.s (K)
-   `T3`: Temperature where viscosity is 1000 Pa.s (K)
-   `T4`: Temperature where viscosity is 10000 Pa.s (K)
-   `T5`: Temperature where viscosity is 100000 Pa.s (K)
-   `T6`: Temperature where viscosity is 1000000 Pa.s (K)
-   `T7`: Temperature where viscosity is 10000000 Pa.s (K)
-   `T8`: Temperature where viscosity is 100000000 Pa.s (K)
-   `T9`: Temperature where viscosity is 1000000000 Pa.s (K)
-   `T10`: Temperature where viscosity is 10000000000 Pa.s (K)
-   `T11`: Temperature where viscosity is 100000000000 Pa.s (K)
-   `T12`: Temperature where viscosity is 1000000000000 Pa.s (K)
-   `Viscosity773K`: Viscosity at 773 K (Pa.s)
-   `Viscosity873K`: Viscosity at 873 K (Pa.s)
-   `Viscosity973K`: Viscosity at 973 K (Pa.s)
-   `Viscosity1073K`: Viscosity at 1073 K (Pa.s)
-   `Viscosity1173K`: Viscosity at 1173 K (Pa.s)
-   `Viscosity1273K`: Viscosity at 1273 K (Pa.s)
-   `Viscosity1373K`: Viscosity at 1373 K (Pa.s)
-   `Viscosity1473K`: Viscosity at 1473 K (Pa.s)
-   `Viscosity1573K`: Viscosity at 1573 K (Pa.s)
-   `Viscosity1673K`: Viscosity at 1673 K (Pa.s)
-   `Viscosity1773K`: Viscosity at 1773 K (Pa.s)
-   `Viscosity1873K`: Viscosity at 1873 K (Pa.s)
-   `Viscosity2073K`: Viscosity at 2073 K (Pa.s)
-   `Viscosity2273K`: Viscosity at 2273 K (Pa.s)
-   `Viscosity2473K`: Viscosity at 2473 K (Pa.s)
-   `Tg`: Glass transition temperature (K)
-   `Tmelt`: Melting temperature (K)
-   `Tliquidus`: Liquidus temperature (K)
-   `TLittletons`: Littletons softening temperature (K)
-   `TAnnealing`: Annealing point (K)
-   `Tstrain`: Strain point (K)
-   `Tsoft`: Softening point (K)
-   `TdilatometricSoftening`: Dilatometric softening temperature (K)
-   `AbbeNum`: Abbe&rsquo;s number
-   `RefractiveIndex`: Refractive index
-   `RefractiveIndexLow`: Refractive index measured at a wavelenght between 0.6 and 1 micron at 293 K
-   `RefractiveIndexHigh`: Refractive index measured at a wavelenght greater than 1 micron at 293 K
-   `MeanDispersion`: Mean dispersion (nF - nC)
-   `Permittivity`: Relative permittivity at ambient temperature anf frequency of1 MHz (or the nearest frequency in the range of 0.01 MHz to 10 MHz)
-   `TangentOfLossAngle`: Tangent of loss angle
-   `TresistivityIs1MOhm.m`: Temperature where the specific electrical resistivity is 1MOhm.m (K)
-   `Resistivity273K`: Specific electrical resistivity measured at 273 K (Ohm.m)
-   `Resistivity373K`: Specific electrical resistivity measured at 373 K (Ohm.m)
-   `Resistivity423K`: Specific electrical resistivity measured at 423 K (Ohm.m)
-   `Resistivity573K`: Specific electrical resistivity measured at 573 K (Ohm.m)
-   `Resistivity1073K`: Specific electrical resistivity measured at 1073 K (Ohm.m)
-   `Resistivity1273K`: Specific electrical resistivity measured at 1273 K (Ohm.m)
-   `Resistivity1473K`: Specific electrical resistivity measured at 1473 K (Ohm.m)
-   `Resistivity1673K`: Specific electrical resistivity measured at 1673 K (Ohm.m)
-   `YoungModulus`: Young&rsquo;s Modulus (GPa)
-   `ShearModulus`: Shear Modulus (GPa)
-   `Microhardness`: Microhardness measured by Knoop or Vickers indentation (GPa)
-   `PoissonRatio`: Poisson&rsquo;s ratio
-   `Density293K`: Density measured at 293 K (g/cm3)
-   `Density1073K`: Density measured at 1073 K (g/cm3)
-   `Density1273K`: Density measured at 1273 K (g/cm3)
-   `Density1473K`: Density measured at 1473 K (g/cm3)
-   `Density1673K`: Density measured at 1673 K (g/cm3)
-   `ThermalConductivity`: Thermal conductivity (W/(m.K))
-   `ThermalShockRes`: Thermal shock resistance (K)
-   `CTEbelowTg`: Linear coefficient of thermal expansion measured below the glass transition temperature (1/K)
-   `CTE328K`: Linear coefficient of thermal expansion measured at 328 +/- 10 K (1/K)
-   `CTE373K`: Linear coefficient of thermal expansion measured at 373 +/- 10 K (1/K)
-   `CTE433K`: Linear coefficient of thermal expansion measured at 433 +/- 10 K (1/K)
-   `CTE483K`: Linear coefficient of thermal expansion measured at 483 +/- 10 K (1/K)
-   `CTE623K`: Linear coefficient of thermal expansion measured at 623 +/- 10 K (1/K)
-   `Cp293K`: Heat capacity at constant pressure measured at 293 K (J/(kg.K))
-   `Cp473K`: Heat capacity at constant pressure measured at 473 K (J/(kg.K))
-   `Cp673K`: Heat capacity at constant pressure measured at 673 K (J/(kg.K))
-   `Cp1073K`: Heat capacity at constant pressure measured at 1073 K (J/(kg.K))
-   `Cp1273K`: Heat capacity at constant pressure measured at 1273 K (J/(kg.K))
-   `Cp1473K`: Heat capacity at constant pressure measured at 1473 K (J/(kg.K))
-   `Cp1673K`: Heat capacity at constant pressure measured at 1673 K (J/(kg.K))
-   `NucleationTemperature`: Nucleation temperature (K)
-   `NucleationRate`: Crystal nucleation rate (1/(s.m3))
-   `TMaxGrowthVelocity`: Temperature of maximum crystal growth velocity (K)
-   `MaxGrowthVelocity`: Maximum crystal growth velocity (m/s)
-   `CrystallizationPeak`: DTA temperature of crystallization peak (K)
-   `CrystallizationOnset`: DTA temperature of crystallization onset (K)
-   `SurfaceTensionAboveTg`: Surface tension above the glass transition temperature (J/m2)
-   `SurfaceTension1173K`: Surface tension at 1173 K (J/m2)
-   `SurfaceTension1473K`: Surface tension at 1473 K (J/m2)
-   `SurfaceTension1573K`: Surface tension at 1573 K (J/m2)
-   `SurfaceTension1673K`: Surface tension at 1673 K (J/m2)



### Estrutura da tabela



O DataFrame está estruturado em dois níveis. No primeiro nível temos informação agrupada por composição, propriedade e metadados.



In [ ]:
df.columns.levels[0]

Veja na célula abaixo como explorar a composição química elementar dos vidros.



In [ ]:
elementos = df["elements"]
elementos

Veja na célula abaixo como explorar uma coluna específica dentro do grupo de propriedades.



In [ ]:
propriedade = df["property"]["Tg"]
propriedade

Com o `pandas` podemos fazer uma análise exploratória dos dados.



In [ ]:
eixo = df["property"]["Tg"].hist()
eixo.set_xlabel("Tg")
eixo.set_ylabel("Quantidade")

In [ ]:
eixo = df["metadata"]["Year"].hist()
eixo.set_xlabel("Ano")
eixo.set_ylabel("Quantidade")

### Controlando a coleta de dados



Na maioria dos casos nós temos interesse em filtrar os dados considerando certas restrições.



In [ ]:
ELEMENTOS_INTERESSE = ["Si", "O", "Ca", "Mg", "K", "Li", "Na", "Al"]
PROPRIEDADE_INTERESSE = "Tg"

In [ ]:
elementos = ["H", "He", "Li", "Be", "B", "C", "N", "O", "F", "Ne",
             "Na", "Mg", "Al", "Si", "P", "S", "Cl", "Ar", "K", "Ca", "Sc",
             "Ti", "V", "Cr", "Mn", "Fe", "Co", "Ni", "Cu", "Zn", "Ga", "Ge",
             "As", "Se", "Br", "Kr", "Rb", "Sr", "Y", "Zr", "Nb", "Mo", "Tc",
             "Ru", "Rh", "Pd", "Ag", "Cd", "In", "Sn", "Sb", "Te", "I", "Xe",
             "Cs", "Ba", "La", "Ce", "Pr", "Nd", "Pm", "Sm", "Eu", "Gd", "Tb",
             "Dy", "Ho", "Er", "Tm", "Yb", "Lu", "Hf", "Ta", "W", "Re", "Os",
             "Ir", "Pt", "Au", "Hg", "Tl", "Pb", "Bi", "Po", "At", "Rn", "Fr",
             "Ra", "Ac", "Th", "Pa", "U", "Np", "Pu"]

elementos_indesejados = elementos.copy()

for item in ELEMENTOS_INTERESSE:
    elementos_indesejados.remove(item)

propriedades_indesejadas = sciglass.available_properties()

propriedades_indesejadas.remove(PROPRIEDADE_INTERESSE)

In [ ]:
config_elementos = {
    "dropline": elementos_indesejados,
    "return_weight": False,
    "final_sum": 100,
}

config_propriedades = {
    "keep": [PROPRIEDADE_INTERESSE],
    "dropline": propriedades_indesejadas,
}

config_compostos = {
    "return_weight": False,
    "final_sum": 100,
    "dropline": elementos,
}

sg = SciGlass(
    elements_cfg=config_elementos,
    properties_cfg=config_propriedades,
    compounds_cfg=config_compostos,
)

sg.remove_duplicate_composition(
    scope="compounds",
    decimals=1,
    aggregator="median",
)

df = sg.data.droplevel(0, axis=1)
df

O processo acima já remove dados duplicados (problema do vazamento de dados).



### Exportando o DataFrame para uma tabela de Excel



In [ ]:
NOME_DO_ARQUIVO = "minha_tabela"

df.to_excel(f"{NOME_DO_ARQUIVO}.xlsx")

### Separando os atributos e target



In [ ]:
X = df.drop(PROPRIEDADE_INTERESSE, axis=1)
y = df.reindex([PROPRIEDADE_INTERESSE], axis=1)

Vamos ver o que obtivemos.



In [ ]:
X

In [ ]:
y

### Divisão dos dados em treino e teste



Para medir a performance de um modelo, precisamos de dados que ele nunca tenha visto. A estratégia abaixo é a conhecida como split de treino e teste.



In [ ]:
TAMANHO_TESTE = 0.1        # fração de dados reservada para teste
SEMENTE_ALEATORIA = 61455  # escolha um número inteiro positivo qualquer

indices = df.index
indices_treino, indices_teste = train_test_split(
    indices, test_size=TAMANHO_TESTE, random_state=SEMENTE_ALEATORIA
)

df_treino = df.loc[indices_treino]
df_teste = df.loc[indices_teste]

X_treino = df_treino.drop(PROPRIEDADE_INTERESSE, axis=1)
y_treino = df_treino.reindex([PROPRIEDADE_INTERESSE], axis=1)

X_teste = df_teste.drop(PROPRIEDADE_INTERESSE, axis=1)
y_teste = df_teste.reindex([PROPRIEDADE_INTERESSE], axis=1)

Vamos ver o que obtivemos para os atributos e target de treino.



In [ ]:
X_treino

In [ ]:
y_treino

Vamos ver o que obtivemos para os atributos e target de teste.



In [ ]:
X_teste

In [ ]:
y_teste

Por fim, agora que terminamos de usar o `pandas`, vamos converter estas variáveis em arrays de `numpy`.



In [ ]:
X_treino = X_treino.values
y_treino = y_treino.values.ravel()
X_teste = X_teste.values
y_teste = y_teste.values.ravel()

<hr>



## Parte 2 - Treinando um modelo



1.  Existem diversos algoritmos de aprendizado de máquina supervisionado

2.  Existem dois tipos de problemas supervisionados: regressão e classificação

3.  Durante o treino de um modelo, o algoritmo é apresentado aos dados de treino (tanto atributos quanto target)

4.  A performance do modelo treinado pode ser estimada utilizando os dados de teste

5.  Uma métrica usual para estimar a performance de modelos de regressão é o RMSE

$$
\mathrm{RMSE} = \sqrt{\sum_{i=1}^{N} \frac{(y_i - \hat{y}_i)^2}{N}}.
$$



### Treinando um modelo preditivo tipo árvore de decisão



#### O que é uma árvore de decisão?



1.  Árvores de decisão são grafos direcionais acíclicos.

2.  Árvores de decisão são modelos interpretáveis!



In [ ]:
modelo = DecisionTreeRegressor(max_depth=3)
modelo.fit(X_treino, y_treino)

figura, eixo = plt.subplots(dpi=300)

tree.plot_tree(
    modelo,
    feature_names=X.columns.tolist(),
    ax=eixo,
    impurity=False,
    filled=True,
    proportion=True,
    precision=2,
)

print("Minha árvore de decisão:")

Podemos representar as decisões na forma de texto também.



In [ ]:
texto = export_text(
    modelo,
    feature_names=X.columns.tolist(),
    decimals=2,
)

print(texto)

#### Treinando uma árvore de decisão



Vamos treinar uma árvore de decisão.



In [ ]:
modelo = DecisionTreeRegressor()
modelo.fit(X_treino, y_treino)

Vamos estimar a performance do modelo.



In [ ]:
y_verdadeiro = y_teste
y_previsao = modelo.predict(X_teste)

RMSE = mean_squared_error(y_verdadeiro, y_previsao, squared=False)

print(f"O RMSE do modelo árvore de decisão foi de {RMSE} unidades de y.")

Uma forma de visualizar a performance é graficando o valor real versus o predito do target.



In [ ]:
figura, eixo = plt.subplots()

x_plot = y_verdadeiro
y_plot = y_previsao

min_ = min(min(x_plot),min(y_plot))
max_ = max(max(x_plot),max(y_plot))

eixo.plot([min_,max_], [min_,max_], label="Identidade")

eixo.plot(
    x_plot,
    y_plot,
    marker="o",
    linestyle="none",
)

eixo.legend()

eixo.set_xlabel("y verdadeiro")
eixo.set_ylabel("y previsto")

### Treinando um modelo preditivo tipo floresta aleatória



O que é uma floresta aleatória?

Vamos treinar uma floresta aleatória.



In [ ]:
modelo = RandomForestRegressor(random_state=61455)
modelo.fit(X_treino, y_treino)

Será que a performance é melhor do que a da árvore de decisão?



In [ ]:
y_verdadeiro = y_teste
y_previsao = modelo.predict(X_teste)

RMSE = mean_squared_error(y_verdadeiro, y_previsao, squared=False)

print(f"O RMSE do modelo floresta aleatória foi de {RMSE} unidades de y.")

In [ ]:
figura, eixo = plt.subplots()

x_plot = y_verdadeiro
y_plot = y_previsao

min_ = min(min(x_plot),min(y_plot))
max_ = max(max(x_plot),max(y_plot))

eixo.plot([min_,max_], [min_,max_], label="Identidade")

eixo.plot(
    x_plot,
    y_plot,
    marker="o",
    linestyle="none",
)

eixo.legend()

eixo.set_xlabel("y verdadeiro")
eixo.set_ylabel("y previsto")

<hr>



## Parte 3 - Aprendendo com o modelo



1.  Nem todos os modelos são interpretáveis como as árvores de decisão (conceito do modelo &ldquo;caixa-preta&rdquo;)

2.  Existem algoritmos que buscam estimar o impacto de cada atributo nas previsões dos modelos (exemplo: SHAP)

3.  Correlação não implica em causalidade



### Valores SHAP



Antes de seguir em frente, precisamos calcular os valores SHAP para cada material.



In [ ]:
explicador = shap.explainers.Tree(modelo, X)
valores_shap = explicador(X)

### Explicando uma previsão



Podemos avaliar o impacto de cada atributo na previsão do modelo.



In [ ]:
POS = 0

shap.plots.waterfall(valores_shap[POS])

### Os atributos de maior impacto



Podemos também checar quais são os atributos considerados mais importantes.



In [ ]:
shap.plots.bar(valores_shap)

### Buscando correlações (que não são necessariamente causalidades!)



Finalmente, podemos também buscar correlações dentro de todo o conjunto de dados.



In [ ]:
shap.plots.beeswarm(valores_shap)

<hr>



## Parte 4 - GlassNet



1.  Existem alguns modelos prontos (isto é, já treinados) para serem usados.

2.  GlassNet é uma rede neural multitarefa capaz de prever 85 propriedades de vidros e líquidos super-resfriados.

3.  GlassNet é um modelo livre disponível dentro do módulo `GlassPy`.



### Usando um modelo preditivo pronto (GlassNet)



Para usar o modelo, primeiro é necessário criar uma instância dele.



In [ ]:
modelo = GlassNet()

Existem diversas formas de realizar uma previsão.



In [ ]:
composicao = "SiO2"

previsao = modelo.predict(composicao)
previsao.T

In [ ]:
composicao = "Li2O(SiO2)2"

previsao = modelo.predict(composicao)
previsao.T

In [ ]:
composicao = {
    "SiO2": 60,
    "Al2O3": 10,
    "Na2O": 30,
}

previsao = modelo.predict(composicao)
previsao.T

Podemos realizar previsões de mais de um vidro ao mesmo tempo.



In [ ]:
data = [
    [1, 0, 2],
    [0, 1, 2],
    [1, 1, 2],
]

composicoes = pd.DataFrame(data, columns=["Li2O", "Na2O", "SiO2"])
composicoes

In [ ]:
previsao = modelo.predict(composicoes)
previsao.T

Podemos usar um DataFrame também, desde que ele tenha apenas colunas relativas à composição.



In [ ]:
previsao = modelo.predict(X)
previsao.T

### Viscosidade com GlassNet



Viscosidade é uma das propriedades mais importantes para fabricação de vidros pois ela determina diversas temperaturas necessárias para o processo de manufatura.



In [ ]:
composicao = "Li2O(SiO2)2"
temperatura = 1000

previsao = modelo.predict_log10_viscosity(
    T=temperatura,
    composition=composicao,
)
previsao

In [ ]:
composicao = "Li2O(SiO2)2"
temperatura = np.linspace(700, 1300)

previsao = modelo.predict_log10_viscosity(
    T=temperatura,
    composition=composicao,
)

figura, eixo = plt.subplots()

eixo.plot(temperatura, previsao)

eixo.set_xlabel("Temperatura (K)")
eixo.set_ylabel("Viscosidade ($\log_{10}$ de Pa.s)")

<hr>



## Parte 5 - Inverse design



1.  O que é o projeto inverso?

2.  Como podemos resolver este problema?



### Código de base



O código abaixo usa o módulo `pymoo` junto com o modelo GlassNet para buscar novos vidros.

No código abaixo usamos uma busca genética já implementada no `pymoo`.



In [ ]:
class Problema(Problem):
    def __init__(self, busca, modelo):
        num_compostos = len(busca["composição"]["compostos"])
        num_objetivos = len(busca["objetivos"])
        num_restricoes = len(busca.get("restrições", []))

        super().__init__(
            n_var=num_compostos,
            n_obj=num_objetivos,
            n_ieq_constr=num_restricoes,
            xl=busca["composição"]["mínimo"],
            xu=busca["composição"]["máximo"],
        )

        self.busca = busca
        self.modelo = modelo

    def _evaluate(self, entrada, saida, *args, **kwargs):
        compostos = self.busca["composição"]["compostos"]
        soma = self.busca["composição"]["soma"]

        x = to_array(entrada, compostos, rescale_to_sum=soma)

        propriedades = self.modelo.predict(x)

        saida["F"] = []
        for obj, stg in self.busca["objetivos"].items():
            saida["F"].append((-1 * stg * propriedades[obj]).values.ravel())

        if self.busca.get("restrições"):
            saida["G"] = []
            for obj, (tipo, valor) in self.busca["restrições"].items():
                index = compostos.index(obj)
                if tipo in [">", ">="]:
                    saida["G"].append(valor - x[:, index])
                elif tipo in ["<", "<="]:
                    saida["G"].append(x[:, index] - valor)


def busca_glassnet(busca):
    """Realiza o design inverso de um vidro."""

    modelo = GlassNet()

    compostos = busca["composição"]["compostos"]
    previsao = list(busca["objetivos"])
    soma = busca["composição"]["soma"]
    num_geracoes = busca["otimização"].get("num_gerações", 40)
    tam_populacao = busca["otimização"].get("tam_populacao", 40)

    problem = Problema(busca, modelo)

    algorithm = NSGA2(
        pop_size=tam_populacao,
        n_offsprings=int(tam_populacao / 4),
        sampling=FloatRandomSampling(),
        crossover=SBX(prob=0.9, eta=15),
        mutation=PM(eta=20),
        eliminate_duplicates=True,
    )

    termination = get_termination("n_gen", num_geracoes)

    res = minimize(
        problem,
        algorithm,
        termination,
        seed=1,
        save_history=True,
        verbose=True,
    )

    x = to_array(res.X, compostos, rescale_to_sum=soma)
    comp = pd.DataFrame(x, columns=compostos)

    prev = pd.DataFrame(res.F, columns=previsao)
    for nome, stg in busca["objetivos"].items():
        prev[nome] /= -stg

    df = pd.concat((comp, prev), axis=1)

    return df

### Configuração



As configurações fundamentais da busca são feitas na célula abaixo.



In [ ]:
COMPOSTOS = ["SiO2", "Li2O", "K2O", "Na2O", "CaO", "MgO", "Al2O3"]
MIN_COMP = 0
MAX_COMP = 100
SOMA_COMP = 100

### Problema de objetivo único (maximização)



In [ ]:
configuracao_busca = {
    "composição": {
        "compostos": COMPOSTOS,
        "mínimo": MIN_COMP,
        "máximo": MAX_COMP,
        "soma": SOMA_COMP,
    },
    "objetivos": {
        "Microhardness": 1,
    },
    "otimização": {
        "num_gerações": 40,
        "tam_população": 40,
    },
}

In [ ]:
resultado_busca = busca_glassnet(configuracao_busca)

In [ ]:
resultado_busca

### Problema de objetivo único (minimização)



In [ ]:
configuracao_busca = {
    "composição": {
        "compostos": COMPOSTOS,
        "mínimo": MIN_COMP,
        "máximo": MAX_COMP,
        "soma": SOMA_COMP,
    },
    "objetivos": {
        "Microhardness": -1,
    },
    "otimização": {
        "num_gerações": 40,
        "tam_população": 40,
    },
}

In [ ]:
resultado_busca = busca_glassnet(configuracao_busca)

In [ ]:
resultado_busca

### Problema de objetivo único com restrição



In [ ]:
configuracao_busca = {
    "composição": {
        "compostos": COMPOSTOS,
        "mínimo": MIN_COMP,
        "máximo": MAX_COMP,
        "soma": SOMA_COMP,
    },
    "objetivos": {
        "Microhardness": 1,
    },
    "restrições": {
        "SiO2": (">", 60),
    },
    "otimização": {
        "num_gerações": 40,
        "tam_população": 40,
    },
}

In [ ]:
resultado_busca = busca_glassnet(configuracao_busca)

In [ ]:
resultado_busca

### Problema multiobjetivo com restrições



In [ ]:
configuracao_busca = {
    "composição": {
        "compostos": COMPOSTOS,
        "mínimo": MIN_COMP,
        "máximo": MAX_COMP,
        "soma": SOMA_COMP,
    },
    "objetivos": {
        "Microhardness": -1,
        "RefractiveIndex": 1,
        "AbbeNum": 1,
    },
    "restrições": {
        "SiO2": (">", 60),
        "Al2O3": ("<", 10),
    },
    "otimização": {
        "num_gerações": 40,
        "tam_população": 40,
    },
}

In [ ]:
resultado_busca = busca_glassnet(configuracao_busca)

In [ ]:
resultado_busca

<hr>



## Referências



1.  [https://numpy.org/](https://numpy.org/)

2.  [https://scipy.org/](https://scipy.org/)

3.  [https://pandas.pydata.org/](https://pandas.pydata.org/)

4.  [https://matplotlib.org/](https://matplotlib.org/)

5.  [https://scikit-learn.org/](https://scikit-learn.org/)

6.  [https://pymoo.org/](https://pymoo.org/)

7.  [https://shap.readthedocs.io/](https://shap.readthedocs.io/)

8.  [https://glasspy.readthedocs.io/](https://glasspy.readthedocs.io/)

9.  Kaufman, S., Rosset, S., Perlich, C., and Stitelman, O. (2012). Leakage in data mining: Formulation, detection, and avoidance. ACM Trans. Knowl. Discov. Data 6, 15:1-15:21. 10.1145/2382577.2382579.

10. Cassar, D.R. (2023). GlassNet: A multitask deep neural network for predicting many glass properties. Ceramics International. 10.1016/j.ceramint.2023.08.281.

11. Lundberg, S.M., and Lee, S.-I. (2017). A unified approach to interpreting model predictions. Advances in Neural Information Processing Systems 30, 4765–4774.

